# Multi-Gaussian MoCo Unseen-Distribution Novelty Test

A simple baseline novelty detector.
* Problem: After seeing imagenet training set for 500 classes, determine if a new image is in a different class.
* Solution: use a simple Gaussian models for each known class.  Score as the ratio of the probabilities (i.e., difference of log probabilities).
* Performance: if novel/non-novel is 50% mix, average precision is 0.72.

First, load imagenet moco model.


In [ ]:
import os, torch
from cmc.models.resnet import InsResNet50
from torchvision import transforms
from netdissect import tally, runningstats, renormalize, parallelfolder

expdir = 'results/places-moco-multi-gaussian'
def ef(s):
    return os.path.join(expdir, s)

dataset = "imagenet"
model_dataset = "places"
model_dir = "/data/vision/torralba/dissect/novelty/models"
model_name = f"{model_dataset}_moco_resnet50.pth"
model_path = os.path.join(model_dir, model_name)
val_path = f"datasets/{dataset}/val"
train_path = f"datasets/{dataset}/train"

img_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    renormalize.NORMALIZER['imagenet']
])
dsv = parallelfolder.ParallelImageFolders([val_path], transform=img_trans, classification=True)
dst = parallelfolder.ParallelImageFolders([train_path], transform=img_trans, classification=True)
dsm = dict(val=dsv, train=dst)
dp_model = InsResNet50()
checkpoint = torch.load(model_path)
dp_model.load_state_dict(checkpoint['model_ema'])
model = dp_model.encoder.module
model.cuda()
None                                    

Below: make a baseline Gaussian model over the whole imagenet distribution.

In [ ]:
layernum = 7

def batch_features(imgbatch, cls):
    result = model(imgbatch.cuda(), layer=layernum)
    if len(result.shape) == 4:
        result = result.permute(0, 2, 3, 1).reshape(-1, result.shape[1])
    return result

split = 'train'
mcov = tally.tally_covariance(batch_features, dsm[split], num_workers=100, batch_size=512, pin_memory=True,
                             cachefile=ef(f'{dataset}-{split}-layer{layernum}-mcov.npz'))

Now make 1000 "selected class" Gaussian models, grouping all observed classes in one giant gaussian.

In [ ]:
def cond_selclass_features(imgbatch, cls):
    result = model(imgbatch.cuda(), layer=layernum)
    if len(result.shape) == 4:
        cls = cls[:,None,None].expand(result.shape[0],
                result.shape[2], result.shape[3]).reshape(-1)
        result = result.permute(0, 2, 3, 1).reshape(-1, result.shape[1])
    return [(c.item(), result[cls == c]) for c in torch.unique(cls)]

selcov = tally.tally_conditional_covariance(cond_selclass_features, dsm[split], num_workers=100, batch_size=512, pin_memory=True,
                    cachefile=ef(f'{dataset}-{split}-layer{layernum}-condsel-mcov.npz'))


Finally, do a test pass over unseen (val set) examples, and measure average precision.

In [ ]:
from netdissect import pbar

selected_classes = 500

mcov.cuda_()
selcov.to_('cuda')

def logp_score(mcov, feat):
    v = feat - mcov.mean()
    b, _ = torch.lstsq(v.t(), mcov.covariance())
    dot = -(v * b.t())
    return dot.sum(1)

# How much we think something is a member of one of the 500 seen claases
def novelty_score(imgdat):
    rep = model(imgdat.cuda(), layer=layernum)
    bestscore = torch.cat([logp_score(selcov.conditional(cc), rep)[None,:]
                           for cc in range(selected_classes)]).max(0)[0]
    return logp_score(mcov, rep) - bestscore

def batch_score_inliers(imgbatch, c):
    selected = imgbatch[c < selected_classes]
    if not len(selected):
        return None
    return novelty_score(selected)[:,None]

def batch_score_outliers(imgbatch, c):
    selected = imgbatch[c >= selected_classes]
    if not len(selected):
        return None
    return novelty_score(selected)[:,None]

rq_inlier = tally.tally_quantile(batch_score_inliers, dsv, num_workers=100, batch_size=512, pin_memory=True,
                   cachefile=ef(f'{dataset}-{split}-layer{layernum}-bestof-sel{selected_classes}-inlier_rq.npz'))
rq_outlier = tally.tally_quantile(batch_score_outliers, dsv, num_workers=100, batch_size=512, pin_memory=True,
                   cachefile=ef(f'{dataset}-{split}-layer{layernum}-bestof-sel{selected_classes}-outlier_rq.npz'))


In [ ]:
from matplotlib import pyplot as plt
plt.title('Validation set scores')
xrange = torch.linspace(0,1,100)
plt.plot(rq_inlier.quantiles(xrange)[0].numpy(), xrange.numpy(), )
plt.plot(rq_outlier.quantiles(xrange)[0].numpy(), xrange.numpy(), )
plt.ylabel('percentile')
plt.xlabel('score')

In [ ]:
srange = torch.linspace(-75, 350, 100)
true_pos = rq_inlier.normalize(srange[None])[0]
false_pos = rq_outlier.normalize(srange[None])[0]
precision = true_pos / (true_pos + false_pos + 1e-20)
accuracy = (true_pos + (1 - false_pos)) / 2
plt.title("multi-Gaussian model novelty detection using MoCo\nFirst %d imagenet classes vs others" % selected_classes)
plt.plot(srange, true_pos, label="True positives")
plt.plot(srange, false_pos, label="False positives")
plt.plot(srange, precision, label="Precision")
plt.plot(srange, accuracy, label="Accuracy, max=%.3g" % accuracy.max().item())
plt.axhline(y=precision.mean(), color='g', linestyle='--', label="AP=%.3g" % precision.mean().item())
plt.xlabel('log score')
plt.legend()
